## Setup Environment

## Fake News Predictions

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

In [ ]:
import numpy as np
import pandas as pd
import re

import torch
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, AdamW, get_linear_schedule_with_warmup

# Load your data
df = pd.read_excel('/content/train_data_mal_fake_detect2.xlsx')

# Define the new label dictionary for 5 classes
label_dict = {'FALSE': 0, 'HALF TRUE': 1, 'MOSTLY FALSE': 2, 'PARTLY FALSE': 3, 'MOSTLY TRUE': 4}

# Update data filtering based on the new label classes
df = df[df.Label.isin(['FALSE', 'HALF TRUE', 'MOSTLY FALSE', 'PARTLY FALSE', 'MOSTLY TRUE'])]


In [ ]:
# Identify classes with too few instances
class_counts = df['Label'].value_counts()
infrequent_classes = class_counts[class_counts < 2].index.tolist()

# Filter out infrequent classes
df = df[~df['Label'].isin(infrequent_classes)]


In [ ]:
import pandas as pd

# Load your data
df = pd.read_excel('/content/train_data_mal_fake_detect2.xlsx')

# Identify the class label with too few instances
class_counts = df['Label'].str.lower().value_counts()
infrequent_classes = class_counts[class_counts < 2].index.tolist()

# Remove instances with the identified class label
df = df[~df['Label'].str.lower().isin(infrequent_classes)]

# Create a new DataFrame with the same data
df_new = df.copy()

# Display value counts of the 'Label' column in the new DataFrame
print(df_new['Label'].value_counts())


False           1097
MOSTLY FALSE     239
FALSE            149
HALF TRUE        141
PARTLY FALSE      42
Name: Label, dtype: int64


In [ ]:
# Identify the class label with too few instances
class_counts = df['Label'].str.lower().value_counts()
infrequent_classes = class_counts[class_counts < 2].index.tolist()

# Remove the identified class with only one instance
df = df[~df['Label'].str.lower().isin(infrequent_classes)]

# Continue with the rest of your preprocessing code


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# Load your data
df = pd.read_excel('/content/train_data_mal_fake_detect2.xlsx')

# Define the new label dictionary for 5 classes
label_dict = {'FALSE': 0, 'HALF TRUE': 1, 'MOSTLY FALSE': 2, 'PARTLY FALSE': 3, 'MOSTLY TRUE': 1}

# Update data filtering based on the new label classes
df = df[df.Label.isin(['FALSE', 'HALF TRUE', 'MOSTLY FALSE', 'PARTLY FALSE', 'MOSTLY TRUE'])]

# Clean text function
def clean_text(text):
    clean_text = re.sub(r'<.*?>', '', text)
    clean_text = re.sub(r'[^a-zA-Z\s]', '', clean_text)
    clean_text = re.sub(r'\.', '. ', clean_text)
    clean_text = clean_text.lower()
    return clean_text

# Apply the clean_text function to the 'News' column
df['text'] = df['News'].apply(clean_text)

# Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.Label.values,
    test_size=0.15,
    random_state=42,
    stratify=None
)

# Set data_type for train and val samples
df['data_type'] = ['not_set'] * df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

# Tokenizer initialization
tokenizer = BertTokenizer.from_pretrained(
    'distilbert-base-uncased',
    do_lower_case=True,
)

# Tokenize and encode the training data
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type == 'train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

# Tokenize and encode the validation data
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type == 'val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

# Extract input_ids, attention_masks, and labels for training set
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].Label.map(label_dict).values)

# Extract input_ids, attention_masks, and labels for validation set
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type == 'val'].Label.map(label_dict).values)

# Create TensorDatasets for training and validation sets
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In t

In [ ]:

# Rest of your preprocessing code remains unchanged

# Model initialization for 5 classes
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=5,  # Number of classes
    output_attentions=False,
    output_hidden_states=False
)

# Rest of your training code remains unchanged

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
epochs = 5
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)
import numpy as np
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [ ]:


# Updated accuracy_per_class function for 5 classes
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    total_correct = 0
    total_samples = 0

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        class_name = label_dict_inverse[label]

        class_correct = np.sum(y_preds == label)
        class_total = len(y_true)

        total_correct += class_correct
        total_samples += class_total

        class_accuracy = class_correct / class_total if class_total > 0 else 0

        print(f'Class: {class_name}')
        print(f'Accuracy: {class_correct}/{class_total} ({class_accuracy * 100:.2f}%)\n')

    total_accuracy = total_correct / total_samples if total_samples > 0 else 0
    print(f'Total Accuracy: {total_correct}/{total_samples} ({total_accuracy * 100:.2f}%)')


In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in tqdm(dataloader_val):

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [ ]:
batch_size = 16
from torch.utils.data import DataLoader, RandomSampler

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

In [ ]:
epochs = 5  # or the desired number of epochs

for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train,
                        desc='Epoch {:1d}'.format(epoch),
                        leave=False,
                        disable=False)

    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }

        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')

    tqdm.write('\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/23 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 1.119747078937033


  0%|          | 0/2 [00:00<?, ?it/s]

Validation loss: 0.9624741971492767
F1 Score (weighted): 0.46146844660194175


...

Validation loss: 0.8987917304039001
F1 Score (weighted): 0.46146844660194175


Epoch 5:   0%|          | 0/23 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.9606911280880803


  0%|          | 0/2 [00:00<?, ?it/s]

Validation loss: 0.8916285932064056
F1 Score (weighted): 0.46146844660194175


In [ ]:
accuracy_per_class(predictions, true_vals)

Class: MOSTLY TRUE
Accuracy: 0/20 (0.00%)

Class: MOSTLY FALSE
Accuracy: 39/39 (100.00%)

Class: PARTLY FALSE
Accuracy: 0/5 (0.00%)

Total Accuracy: 39/64 (60.94%)


In [ ]:
torch.save(model.state_dict(), f'BERT_ft_Epoch{epoch}.model')

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Replace 'model_name_or_path' with the actual name or path of your pre-trained model
model_name_or_path = 'bert-base-uncased'

# Load the tokenizer and model using from_pretrained
tokenizer = BertTokenizer.from_pretrained(model_name_or_path)
model = BertForSequenceClassification.from_pretrained(model_name_or_path)

# Rest of your code


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Inference code with the loaded model
# Replace 'distilbert-base-uncased' with the appropriate path or name if you have saved the model locally
saved_model_path = 'BERT_ft_Epoch5.model'
loaded_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
loaded_model.classifier = torch.nn.Linear(loaded_model.config.hidden_size, 5)
loaded_model.load_state_dict(torch.load(saved_model_path))

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load test data from Excel
test_data_path = '/content/test-data2.xlsx'
test_data = pd.read_excel(test_data_path)

# Extract 'News' column from the test data
test_sentences = test_data['News'].tolist()

# Perform inference using the loaded model in batches
predicted_labels = []

batch_size = 8
for i in range(0, len(test_sentences), batch_size):
    batch_sentences = test_sentences[i:i + batch_size]

    # Ensure each sentence is a string
    batch_sentences = [str(sentence) for sentence in batch_sentences]

    encoded_test_sentences = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        logits = loaded_model(**encoded_test_sentences.to(loaded_model.device))[0]

    probabilities = torch.softmax(logits, dim=1)
    batch_predicted_labels = torch.argmax(probabilities, dim=1).tolist()

    predicted_labels.extend(batch_predicted_labels)

# Display predicted labels
print("Predicted Labels:", predicted_labels)

# Map numerical labels to text labels
label_dict = {0: 'FALSE', 1: 'HALF TRUE', 2: 'MOSTLY FALSE', 3: 'PARTLY FALSE'}
predicted_labels_text = [label_dict[label] for label in predicted_labels]
test_data['predicted_labels'] = predicted_labels_text

# Save the results to a CSV file
test_data.to_csv('/content/DistilBert_results.csv', index=False)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted Labels: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [ ]:
test_data['predicted_labels'].unique()

array(['MOSTLY FALSE'], dtype=object)